Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://manipulation.csail.mit.edu/intro.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20201117'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py", "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
#proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Imports
import numpy as np
from ipywidgets import ToggleButton, ToggleButtons

import pydrake.all
from pydrake.all import RigidTransform, RotationMatrix, AutoDiffXd, Expression, Variable
from manipulation.scenarios import AddIiwa, AddTwoLinkIiwa

# TODO: Use pydrake.symbolic.MakeVectorVariable once that binding PR lands.
def MakeVectorVariable(name, dim):
    return np.array([pydrake.symbolic.Variable(f"{name}{i}") for i in range(dim)])

# TODO: Use pydrake.symbolic version once it lands.
def DecomposeAffineExpressions(e):
    vars = e[0].GetVariables()
    for ei in e[1:]:
        vars.insert(ei.GetVariables())
    p = list(vars)
    A = pydrake.symbolic.Evaluate(pydrake.symbolic.Jacobian(e, p))
    b = pydrake.symbolic.Evaluate(pydrake.symbolic.Substitute(e, {a:0 for a in p}))
    return A, b

pydrake.common.set_log_level("warn");

In [2]:

def symbolic_dynamics_example():
    plant = pydrake.multibody.plant.MultibodyPlant(1.0)  # time_step doesn't matter for this example.
    iiwa = AddTwoLinkIiwa(plant)
    plant.Finalize()

    sym_plant = plant.ToSymbolic()
    q = MakeVectorVariable("q", 2)
    v = MakeVectorVariable("v", 2)
    tau = MakeVectorVariable("u", 2)
    sym_context = sym_plant.CreateDefaultContext()
    sym_plant.get_actuation_input_port().FixValue(sym_context, tau)
    sym_plant.SetPositions(sym_context, q)
    sym_plant.SetVelocities(sym_context, v)

    vd = MakeVectorVariable("vd", 2)
    forces = pydrake.multibody.tree.MultibodyForces_[pydrake.symbolic.Expression](sym_plant)
    sym_plant.CalcForceElementsContribution(sym_context, forces)
    sym_tau = sym_plant.CalcInverseDynamics(sym_context, vd, forces)
    print(sym_tau[0].Expand())

symbolic_dynamics_example()

(1.0527542065909233e-31 + 0.5 * v0 + 0.17176310450000001 * vd0 - 0.055708182999999994 * vd1 - 2.9400411811014152e-49 * (v0 * v1) - 5.5379690013737233e-48 * (v0 * v1 * sin(q0)) - 5.7765412023497699e-36 * (v0 * v1 * sin(q0) * sin(q1)) - 1.2431249748483347e-47 * (v0 * v1 * sin(q0) * sin(q1) * pow(cos(q0), 2)) - 1.608074870934695e-47 * (v0 * v1 * sin(q0) * sin(q1) * pow(cos(q0), 2) * cos(q1)) - 1.0947644252537633e-47 * (v0 * v1 * sin(q0) * sin(q1) * cos(q1)) + 8.4248122298583822e-49 * (v0 * v1 * sin(q0) * pow(sin(q1), 2)) + 1.29040645071254e-47 * (v0 * v1 * sin(q0) * pow(sin(q1), 2) * pow(cos(q0), 2)) + 8.8949609551868271e-48 * (v0 * v1 * sin(q0) * cos(q0)) + 7.7769284013805058e-64 * (v0 * v1 * sin(q0) * pow(cos(q0), 2)) - 2.3138262180345334e-47 * (v0 * v1 * sin(q0) * pow(cos(q0), 2) * cos(q1)) - 2.237289579953558e-47 * (v0 * v1 * sin(q0) * pow(cos(q0), 2) * pow(cos(q1), 2)) + 6.7181527354843034e-34 * (v0 * v1 * sin(q0) * cos(q1)) + 6.9232624285518144e-47 * (v0 * v1 * sin(q0) * pow(cos(q1)

In [4]:
def symbolic_mustard_example():
    plant = pydrake.multibody.plant.MultibodyPlant(1.0)  # time_step doesn't matter for this example.
    iiwa = AddTwoLinkIiwa(plant)
    parser = pydrake.multibody.parsing.Parser(plant)
    mustard = parser.AddModelFromFile(pydrake.common.FindResourceOrThrow(
        "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf"))
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_ee", iiwa), 
                    plant.GetFrameByName("base_link_mustard", mustard))
    plant.Finalize()
    context = plant.CreateDefaultContext()

    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()
    # TODO: Remove this line on resolution of #14344.
    sym_plant.SetPositions(sym_context, plant.GetPositions(context))

    sym_inertia = pydrake.multibody.tree.SpatialInertia_[Expression](
        1.0, 
        MakeVectorVariable("m_com", 3), 
        pydrake.multibody.tree.UnitInertia_[Expression](Variable("Ixx"), Variable("Iyy"), Variable("Izz"), Variable("Ixy"), Variable("Ixz"), Variable("Iyz")))
    body = sym_plant.GetBodyByName("base_link_mustard")
    body.SetSpatialInertiaInBodyFrame(sym_context, sym_inertia)

    vd = np.random.randn(2).astype(Expression)
    forces = pydrake.multibody.tree.MultibodyForces_[pydrake.symbolic.Expression](sym_plant)
    sym_plant.CalcForceElementsContribution(sym_context, forces)
    sym_tau = sym_plant.CalcInverseDynamics(sym_context, vd, forces)
    print(f"tau[0] = {sym_tau[0].Expand()}")
    print(f"tau[1] = {sym_tau[1].Expand()}")

    w, y = DecomposeAffineExpressions(sym_tau)

    print(f"w = {w}")
    print(f"y = {y}")

symbolic_mustard_example()

tau[0] = (-31.437714139650371 - 8.4132135365686072 * m_com0 + 3.1573901569485536 * m_com1 - 1.5456650091802271e-11 * m_com2 + 3.0460567470935521e-31 * Ixx + 1.8965929950818608e-23 * Iyy + 0.79085991989924953 * Izz + 4.81637418820688e-27 * Ixy + 9.8352063567868261e-16 * Ixz + 7.7458101830234417e-12 * Iyz)
tau[1] = (27.143453259514324 + 8.5335747109343316 * m_com0 - 2.8478029670627292 * m_com1 + 1.3940987770061476e-11 * m_com2 - 1.766121271181897e-31 * Ixx - 1.8963166010672108e-23 * Iyy - 0.79085991989924953 * Izz - 3.7414815094795156e-27 * Ixy - 7.6408681437263572e-16 * Ixz - 7.7452457659640215e-12 * Iyz)
w = [[-8.41321354e+00  3.15739016e+00 -1.54566501e-11  3.04605675e-31
   1.89659300e-23  7.90859920e-01  4.81637419e-27  9.83520636e-16
   7.74581018e-12]
 [ 8.53357471e+00 -2.84780297e+00  1.39409878e-11 -1.76612127e-31
  -1.89631660e-23 -7.90859920e-01 -3.74148151e-27 -7.64086814e-16
  -7.74524577e-12]]
y = [[-31.43771414]
 [ 27.14345326]]


In [ ]:
def parameter_estimation_example():
    plant = pydrake.multibody.plant.MultibodyPlant(time_step=0.002)  
    iiwa = AddIiwa(plant)
    parser = pydrake.multibody.parsing.Parser(plant)
    mustard = parser.AddModelFromFile(pydrake.common.FindResourceOrThrow(
        "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf"))
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_ee", iiwa), 
                    plant.GetFrameByName("base_link_mustard", mustard))
    plant.Finalize()
    context = plant.CreateDefaultContext()

    ad_plant = plant.ToAutoDiffXd()
    ad_context = ad_plant.CreateDefaultContext()

    # TODO: Remove this line on resolution of #14344.
    ad_plant.SetPositions(ad_context, plant.GetPositions(context))

    # initial guess
    mass = [1.0]
    com = [0, 0, 0]
    unit_inertia = [1, 1, 1, 0, 0, 0]

    ad_mass, ad_com, ad_unit_inertia = pydrake.autodiffutils.initializeAutoDiffTuple(mass, com, unit_inertia)
    ad_inertia = pydrake.multibody.tree.SpatialInertia_[AutoDiffXd](
        ad_mass[0][0], 
        ad_com[:,0], 
        pydrake.multibody.tree.UnitInertia_[AutoDiffXd](*ad_unit_inertia[:,0].tolist()))
    body = ad_plant.GetBodyByName("base_link_mustard")
    body.SetSpatialInertiaInBodyFrame(ad_context, ad_inertia)

    forces = pydrake.multibody.tree.MultibodyForces_[AutoDiffXd](ad_plant)
    ad_plant.CalcForceElementsContribution(ad_context, forces)
    vd = [0]*7
    tau = ad_plant.CalcInverseDynamics(ad_context, vd, forces)
    print(tau)

parameter_estimation_example()

In [ ]:
# TODO: make the data, by waving the arm around a bit.  animate it in meshcat.
# TODO: mathematical program with mass, com, unit_inertia as decision variables.  The costs are linear in com / inertia, so add them as symbolic objectives for sure! 